The following is a d3 graph-like tree example. To run the program, you must execute each cell one at a time. If you get the error "d3 not loaded", rerun the first cell (%load_ext py_d3)

In [10]:
%load_ext py_d3

The py_d3 extension is already loaded. To reload it, use:
  %reload_ext py_d3


In [13]:
%%d3 4.2.2
<style>
        .node circle {
          fill: #fff;
          stroke: steelblue;
          stroke-width: 3px;
        }

        .node text {
          font: 12px sans-serif;
        }

        .link {
          fill: none;
          stroke: #ccc;
          stroke-width: 2px;
        }

</style>

<div id="key">
        <p> triangles have children, circles no children </p>
        <p> dashed: synchronous (direct) </p>
        <p> solid: asynchronous (normal) </p>
    </div>
<g></g>

<script>
function parseNewick(a){for(var e=[],r={},s=a.split(/\s*(;|\(|\)|,|:)\s*/),t=0;t<s.length;t++){var n=s[t];switch(n){case"(":var c={};r.branchset=[c],e.push(r),r=c;break;case",":var c={};e[e.length-1].branchset.push(c),r=c;break;case")":r=e.pop();break;case":":break;default:var h=s[t-1];")"==h||"("==h||","==h?r.name=n:":"==h&&(r.length=parseFloat(n))}}return r}

// Set the dimensions and margins of the diagram
var margin = {top: 20, right: 90, bottom: 30, left: 90},
    width = 560 - margin.left - margin.right, //make respondant to ipynb width
    height = 300 - margin.top - margin.bottom;
    
var svg = d3.select("g").append("svg")
    .attr("width", width + margin.right + margin.left)
    .attr("height", height + margin.top + margin.bottom)
    .append("g")
    .attr("transform", "translate("
          + margin.left + "," + margin.top + ")");
var i = 0,
    duration = 750,
    root;
    
var treemap = d3.tree().size([height, width]);

var treemat = [1];

d3.queue()
        .defer(d3.text, "static/treeformat.txt")
        .defer(d3.csv, "static/perfdata.csv")
        .await(analyze);   
function analyze(error, treeformat, perfdata) {
    if (error) throw error;
    // Assigns parent, children, height, depth
    root = d3.hierarchy(parseNewick(treeformat), function(d) { return d.branchset; });
    root.x0 = height / 2;
    root.y0 = 0;

    // Collapse after the second level
    //root.children.forEach(collapse); //katy
    
    // Collecting the performance times
    domainTimes = [];
    prim_inst = [];
    perfdata.map(function(d){
        var avgTime = ((+d.time) + (+d.direct_time))/(d.count + d.direct_count);
        if (avgTime > 0) {
            domainTimes.push(avgTime * 1.e-9);
        }
        
        prim_inst.push(d.primitive_instance);
    });
    var colors = ["#edf8fb", "#b2e2e2", "#66c2a4", "#2ca25f", "#006d2c"];
    colorTimeScale = d3.scaleQuantize() //went with log scale
                .domain(d3.extent(domainTimes)) //need to put scale on graph
                .range(colors);
    
    update(root, perfdata);
    addLegend(d3.extent(domainTimes), colors);
    count = 1; //count the root
    countNodes(root);
}
// Collapse the node and all it's children
function collapse(d) {
  if(d.children) {
    d._children = d.children;
    d._children.forEach(collapse);
    d.children = null;
  }
}

function addLegend(domainTimes, colors) {
    var formatPercent = d3.format(".0%"),
    formatNumber = d3.format(".0f");

//var threshold = d3.scaleThreshold()
//    .domain([0.11, 0.22, 0.33, 0.50])
//    .range(["#6e7c5a", "#a0b28f", "#d8b8b3", "#b45554", "#760000"]);
//gotta get the domain values for each color
var domainVals = [colorTimeScale.invertExtent(colors[0])[0]];
var greatestVal = colorTimeScale.invertExtent(colors[colors.length-1])[1];
for (var i=0; i<colors.length; i++){
    domainVals.push(colorTimeScale.invertExtent(colors[i])[1]);
}

var x = d3.scaleLinear()
    .domain([0, greatestVal])
    .range([0, 200]);

var xAxis = d3.axisBottom(x)
    .tickSize(13)
    .tickValues(domainVals);
    //.tickFormat(function(d) { return d === 0.5 ? formatPercent(d) : formatNumber(100 * d); });

var g = d3.select("#key").append("svg")
    .attr("width", 50 + margin.right + margin.left)
    .attr("height", 15 + margin.bottom)
  .append("g")
    .attr("transform", "translate(16,17)")
          //+ margin.left + "," + margin.top + ")")
          .call(xAxis);

g.select(".domain")
    .remove();

g.selectAll("rect")
  .data(colorTimeScale.range().map(function(color) {
    var d = colorTimeScale.invertExtent(color);
    if (d[0] == null) d[0] = x.domain()[0];
    if (d[1] == null) d[1] = x.domain()[1];
    return d;
  }))
  .enter().insert("rect", ".tick")
    .attr("height", 8)
    .attr("x", function(d) { return x(d[0]); })
    .attr("width", function(d) { return x(d[1]) - x(d[0]); })
    .attr("fill", function(d) { return colorTimeScale(d[0]); });

g.append("text")
    .attr("fill", "#000")
    .attr("font-weight", "bold")
    .attr("text-anchor", "start")
    .attr("y", -6)
    .text("Average time per instance");   
}

// Count the number of nodes at each level
function countNodes(node) {
    if(node.children){
        for(var i=0; i<node.children.length; i++) {
            if (node.children[i].children){
                count++;
                countNodes(node.children[i]);
            }
            else {
                count++;
            }
        }
    }
}


function update(source, perfdata) {

  // Assigns the x and y position for the nodes
  var tree = treemap(root);

  // Compute the new tree layout.
  var nodes = tree.descendants(),
      links = tree.descendants().slice(1);
  var maxDepth=0; 
  nodes.forEach(function(d){
      if (d.depth > maxDepth){ 
        maxDepth = d.depth;
      } 
  });

  // Normalize for fixed-depth. 
  nodes.forEach(function(d){ 
      depthY = width / maxDepth;
      d.y = d.depth * depthY;
  }); //was 180. Made dependent on # of levels and width

  // ****************** Nodes section ***************************

  // Update the nodes...
  var node = svg.selectAll('g.node')
      .data(nodes, function(d) {
          var nodename = d.data.name;
          // Change time to time-per-instance //katy
          if (prim_inst.indexOf(nodename) >= 0) {
            node_perfdata = perfdata[prim_inst.indexOf(nodename)];    
            node_perfdata.avg_time = (node_perfdata.time / node_perfdata.count) * 1.e-9; //katy nanos are SMALL
            node_perfdata.avg_direct_time = (node_perfdata.direct_time / node_perfdata.direct_count) * 1.e-9;
            d._perfdata = perfdata[prim_inst.indexOf(nodename)];  
          }
          return d.id || (d.id = ++i); 
          });

  // Enter any new modes at the parent's previous position.
  var nodeEnter = node.enter().append('g')
      .attr('class', 'node')
      .attr("transform", function(d) {
        return "translate(" + source.y0 + "," + source.x0 + ")";
    })  
    .on('click', click) 
    //.on("mousemove", tool_tip.hide) 
    /* //tooltip
    .on("mouseenter", tool_tip.show)
    .on("mouseout", tool_tip.hide);
    */;

  // Add Circle for the nodes
//  nodeEnter.append('circle')
//      .attr('class', 'node')
//      .attr('r', 10)
//      .style("stroke", function(d){ //stroke color by direct or not
//          if (d._perfdata) {
//              return (d._perfdata.avg_direct_time > 0) ? "red" : "purple"; 
//          }
//      });

// Add Triangle for nodes when collapsed and have children
//katy

var symbol = d3.symbol().size([150]);
nodeEnter
        .append("path")
        .attr("class", "node")
        .attr("d", symbol.type(function(d){
            if (d.children) {
                return d3.symbolTriangle;
            } else {
                return d3.symbolCircle;
            }
        }))
        .attr('transform', "rotate(-90)")
        .attr("stroke-dasharray", function(d){ //stroke solid/dashed by direct or not
          if (d._perfdata) {
              return (d._perfdata.avg_direct_time > 0) ? "3, 3" : "0"; 
          }
          return "0";
        })
        .style("stroke", "black")
        .style("fill", function(d) { //katy
        if (d._perfdata) { //color circle by time-per-instance
            if (d._perfdata.avg_time > 0) {
                return colorTimeScale(d._perfdata.avg_time);
            }
            else if (d._perfdata.avg_direct_time > 0) {
                return colorTimeScale(d._perfdata.avg_direct_time);
            }
            return "#fff";
        }
        return "#fff";
    });


// Add labels for the nodes
  nodeEnter.append('text')
      .attr("dy", ".35em")
      .attr("x", function(d) {
          return d.children || d._children ? -13 : 13;
      })
      .attr("text-anchor", function(d) {
          return d.children || d._children ? "end" : "start";
      })
      .text(function(d) { 
          if (!d.children) {
            return d.data.name.substring(9); 
          } else {
              return "";
          }
      });

  // UPDATE
  var nodeUpdate = nodeEnter.merge(node);
  
  // Transition to the proper position for the node
  nodeUpdate.transition()
    .duration(duration)
    .attr("transform", function(d) { 
        return "translate(" + d.y + "," + d.x + ")";
     });

  // Update the node attributes and style
  nodeUpdate.select('circle.node')
    .attr('r', 10)
    .style("fill", function(d) { //katy
        if (d._perfdata) { //color circle by time-per-instance
            if (d._perfdata.avg_time > 0) {
                return colorTimeScale(d._perfdata.avg_time);
            }
            else if (d._perfdata.avg_direct_time > 0) {
                return colorTimeScale(d._perfdata.avg_direct_time);
            }
        }
        //return "#fff";
        //return d._children ? "lightsteelblue" : "#fff";
    })
    .attr('cursor', 'pointer');

  // Remove any exiting nodes
  var nodeExit = node.exit().transition()
      .duration(duration)
      .attr("transform", function(d) {
          return "translate(" + source.y + "," + source.x + ")";
      })
      .remove();

  // On exit reduce the node circles size to 0
  nodeExit.select('circle')
    .attr('r', 1e-6);

  // On exit reduce the opacity of text labels
  nodeExit.select('text')
    .style('fill-opacity', 1e-6);

  // ****************** links section ***************************

  // Update the links...
  var link = svg.selectAll('path.link')
      .data(links, function(d) { return d.id; });

  // Enter any new links at the parent's previous position.
  var linkEnter = link.enter().insert('path', "g")
      .attr("class", "link")
      .attr('d', function(d){
        var o = {x: source.x0, y: source.y0};
        return diagonal(o, o);
      });
// UPDATE
  var linkUpdate = linkEnter.merge(link);

  // Transition back to the parent element position
  linkUpdate.transition()
      .duration(duration)
      .attr('d', function(d){ return diagonal(d, d.parent) });

  // Remove any exiting links
  var linkExit = link.exit().transition()
      .duration(duration)
      .attr('d', function(d) {
        var o = {x: source.x, y: source.y};
        return diagonal(o, o);
      })
      .remove();

  // Store the old positions for transition.
  nodes.forEach(function(d){
    d.x0 = d.x;
    d.y0 = d.y;
  });

  // Creates a curved (diagonal) path from parent to the child nodes
  function diagonal(s, d) {

    path = `M ${s.y} ${s.x}
            C ${(s.y + d.y) / 2} ${s.x},
              ${(s.y + d.y) / 2} ${d.x},
              ${d.y} ${d.x}`;

    return path;
  }

  // Toggle children on click.
  function click(d) {
    if (d.children) {
        d._children = d.children;
        d.children = null;
      } else {
        d.children = d._children;
        d._children = null;
      }
    update(d, perfdata);
  }
}     
</script>